In [22]:
import cv2
import imutils
# from imutils.video import VideoStream
import dlib
import scipy
import numpy as np
'''
All packages imported successfully.
'''

'\nAll packages imported successfully.\n'

## LAB color space
#### L* for the lightness from black (0) to white (100), a* from green (−) to red (+), and b* from blue (−) to yellow (+).

In [4]:
# print("[INFO] starting video stream thread...")
# vs = VideoStream(src=arg["webcam"]).start()
# time.sleep(1.0)
 
# # loop over frames from the video stream
# while True:
# 	# grab the frame from the threaded video file stream, resize
# 	# it, and convert it to grayscale
# 	# channels)
# 	frame = vs.read()
# 	frame = imutils.resize(frame, width=450)
# 	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
# 	# detect faces in the grayscale frame
# 	rects = detector(gray, 0)
    
# cv2.imshow("output",frame)
# cv2.waitKey(0)
# cv2.destroyWindow('img')

In [23]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyWindow('frame')

#### Processing image filtering and contrast on video frames

### Remove lighting from background to get clear face image

In [24]:
def light_remover(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lab =  cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    L = lab[:,:,0]
#     l,a,b = cv2.split(lab)
    # Apply CLAHE technique to improve contrast
#     clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(12,12))
#     cl = clahe.apply(l)
    # median blur to remove salt and pepper noise
#     l = cv2.medianBlur(l,99)
#     lab = cv2.merge((cl,a,b)) # merge the CLAHE applied 'L' property 
    composed = cv2.cvtColor(lab,cv2.COLOR_LAB2BGR)
    # invert the image lighting
    Medianblur = cv2.medianBlur(L,99)
    invertedL = cv2.bitwise_not(Medianblur)
    composed = cv2.addWeighted(gray,0.80,invertedL,0.20,0)
    return composed
    

In [25]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
#     lab = cv2.cvtColor(frame,cv2.COLOR_BGR2LAB)
#     l,a,b = cv2.split(lab)
#     clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(12,12))
#     cl = clahe.apply(l)
#     limg = cv2.merge((cl,a,b))
#     final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
# #     Try median filtering to remove noise
#     final = cv2.medianBlur(final,3)
    final  = light_remover(frame)
    
    # Display the resulting frame
    cv2.imshow('frame',final)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyWindow('frame')

In [16]:
cap.release()

### Using Thread based WebcamVideoStream object of imutils to improve performance

In [22]:
from imutils.video import WebcamVideoStream
from imutils.video import FPS
import cv2
import time

print("Starting Video stream...")
# vs = VideoStream(resolution=(640, 480), framerate=32).start()
vs = WebcamVideoStream().start()
time.sleep(1.0)
# Start FPS timer
fps = FPS().start()

while True:
    frame = vs.read()
    final = light_remover(frame)

    cv2.imshow("frame", final)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    fps.update()

fps.stop()
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
vs.stream.release()
vs.stop()
cv2.destroyWindow('frame')


Starting Video stream...
[INFO] approx. FPS: 7.23


# Frustrated attempt.. Pycharm execution gives 30fps approx.

In [23]:
from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import imutils
import time
import cv2


def light_remover(frame):
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    #     L = lab[:,:,0]
    l, a, b = cv2.split(lab)
    # Apply CLAHE technique to improve contrast
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(12, 12))
    cl = clahe.apply(l)
    # median blur to remove salt and pepper noise
    l = cv2.medianBlur(l, 99)
    lab = cv2.merge((cl, a, b))  # merge the CLAHE applied 'L' property
    composed = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    # invert the image lighting
    #     Medianblur = cv2.medianBlur(L,99)
    #     invertedL = cv2.bitwise_not(Medianblur)
    #     composed = cv2.addWeighted(gray,0.80,invertedL,0.20,0)
    return composed


vs = FileVideoStream(0).start()
time.sleep(1.0)
# Start FPS Timer
fps = FPS().start()

while True:

    Frame = vs.read()
#     process the frame and store it in the final stack
    final = np.dstack([light_remover(Frame)])
    # final = cv2.cvtColor(final, cv2.COLOR_BGR2GRAY)
    final = np.dstack([final])
#     print(final.shape)
    cv2.imshow("Frame", final)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    fps.update()


fps.stop()
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
cv2.destroyWindow('frame')
vs.stop()


[INFO] approx. FPS: 6.86


Anaconda `[INFO] approx. FPS: 6.58`

#### Execution from PyCharm
C:\Users\Amit\Anaconda3\envs\Driver_Alert_Drowsiness_Yawn\python.exe `C:/Users/Amit/PycharmProjects/Driver_Alert_Drowsiness_Yawn/Process_Webcam_Thread.py` [INFO] approx. `FPS: 30.54`